In [2]:
from pyscf import gto, scf, dft
from pyscf.lo import orth


import numpy as np
from IPython.display import clear_output
import pandas as pd
from pathlib import Path


In [4]:
def np_to_scfinput(atoms, coords):
    '''
    Input: tuple of (coordinates: np.array; atom representation: tuple of strings)
    '''
    # safeguarding here
    # convertion
    raw = []
    for atom, coord in zip(atoms, coords):
        raw.append(' '.join([atom, *coord.astype(str)]))
    
    return ' ;'.join(raw)

def add_gaussian_noise(coords, sigma=0.02, seed=None):
    '''
    Input: np array shape (N,3)
    First row is central atom: 0,0,0. Or any other atom that is kept at the origin
    Second and third rows are hydrogens / all other atoms
    '''
    # default_rng is the new np random Generator. good to use!
    rng = np.random.default_rng(seed)
    noise = rng.normal(loc=0.0, scale=sigma, size=coords.shape)
    coords_w_noise = coords + noise

    # force the first row (central atom) to be 0,0,0
    coords_w_noise[0] = [0,0,0]
    return coords_w_noise

def scf_features(mf, mol):

    # Grab SAAO coefficients from a molecule
    S = mf.get_ovlp()                               # AO overlap (nao×nao)
    C_sao = orth.orth_ao(mol, method='lowdin', s=S)     # Symmetric orthogonalizer: for transformation to SAAO basis

    # Obtain the AO basis matrices:
    H_ao  = mf.get_hcore()                              # Core Hamiltonian in AO basis
    F_ao  = mf.get_fock(dm=mf.make_rdm1())              # Fock in AO basis
    P_ao  = mf.make_rdm1()                              # Density in AO basis
    J_ao, K_ao = mf.get_jk(mf, dm=mf.make_rdm1())

    # Project into SAAO basis:
    H_sao = C_sao.T @ H_ao @ C_sao
    F_sao = C_sao.T @ F_ao @ C_sao
    P_sao = C_sao.T @ P_ao @ C_sao
    J_sao = C_sao.T @ J_ao @ C_sao
    K_sao = C_sao.T @ K_ao @ C_sao

    # diagonal blocks per atom:
    diag_blocks = []
    atom_slices = mol.aoslice_by_atom()
    for (_, _, ist, sto) in atom_slices:
        diag_blocks.append(np.stack([
        np.diag(F_sao[ist:sto, ist:sto]),
        np.diag(J_sao[ist:sto, ist:sto]),
        np.diag(K_sao[ist:sto, ist:sto]),
        np.diag(P_sao[ist:sto, ist:sto]),
        np.diag(H_sao[ist:sto, ist:sto]),
        ], axis=1))

    # Pad each block to (max_n,5)
    max_n = max(b.shape[0] for b in diag_blocks)
    padded = []
    for b in diag_blocks:
        pad = np.zeros((max_n - b.shape[0], 5), dtype=b.dtype)
        padded.append(np.vstack([b, pad]))

    return np.stack(padded, axis=0)  # shape (natoms, max_n, 5)


## Prepare water molecule file (default)

1. Water molecule

In [22]:
# actual water molecule data
bond_len = 0.9572
bond_ang = 104.5
atoms = ('O','H','H') # what atom that each row corresponds to 

2. $CO_2$ molecule

In [5]:
bond_len = 1.163
bond_ang = 180
atoms = ('C','O','O') # what atom that each row corresponds to 


In [23]:
x = bond_len * np.sin(np.radians(bond_ang))
y = bond_len * np.cos(np.radians(bond_ang))

coords = np.array([[0,0,0],[x, y, 0],[-x,y,0]]) # put in the actual data
print(coords)

[[ 0.          0.          0.        ]
 [ 0.92671092 -0.23966374  0.        ]
 [-0.92671092 -0.23966374  0.        ]]


3. More complex CH3OH molecule: 

In [10]:

atoms = ('C', 'H', 'H', 'H', 'O', 'H')

coords = np.array([
    [ 0.0000,   0.0000,   0.0000],   # C
    [ 0.4970,   0.0280,   0.9736],   # H
    [ 0.2846,   0.8860,  -0.5809],   # H
    [ 0.3124,  -0.9001,  -0.5437],   # H
    [-1.3613,  -0.0150,   0.3035],   # O
    [-1.8357,  -0.0394,  -0.5177],   # H
])

The code

In [24]:
# change this:
#####################
data_dir = Path('./h2o_energies_noise_scf_features') # change this
file_name = 'h2o_noise_scf_dft_b3lyp_def2-SVP'
# mf_scanner = gto.M().apply(dft.RKS).set(xc='b3lyp', basis='def2-SVP').as_scanner() # an easy way to ask for energy only
####################

data_dir.mkdir(exist_ok=True)

sigma = 0.02

for i in range(1000):
    coords_w_noise = add_gaussian_noise(coords, sigma=sigma, seed=i)
    inp = np_to_scfinput(atoms, coords_w_noise)
    mol = gto.Mole(atom=inp, basis='def2-SVP', charge=0, spin=0)
    rhf = scf.RHF(mol)
    e = rhf.kernel()
    features = scf_features(rhf, mol)


    np.savez_compressed(
        data_dir / f'{file_name}_{i}.npz', # change y
        coords=coords_w_noise,
        energy = e,
        noise_sigma = sigma,
        features = features
    )

Initialize <pyscf.gto.mole.Mole object at 0x7162560cd4f0> in <pyscf.scf.hf.RHF object at 0x716258700530>


converged SCF energy = -75.9247089792245


Initialize <pyscf.gto.mole.Mole object at 0x7162560cf2c0> in <pyscf.scf.hf.RHF object at 0x716255f6d910>


converged SCF energy = -75.9231085282139


Initialize <pyscf.gto.mole.Mole object at 0x716255e23b90> in <pyscf.scf.hf.RHF object at 0x716255e20530>


...

converged SCF energy = -75.9261254383056


Initialize <pyscf.gto.mole.Mole object at 0x716255e159d0> in <pyscf.scf.hf.RHF object at 0x716255e162d0>


converged SCF energy = -75.9262870060949


Initialize <pyscf.gto.mole.Mole object at 0x716255ef5910> in <pyscf.scf.hf.RHF object at 0x716255f6e1e0>


converged SCF energy = -75.9240013496878


In [21]:
# inspection of one single file
index = 6
file_path = data_dir / f'{file_name}_{index}.npz'
data = np.load(file_path)
print(
    data['coords'], data['energy'], data['noise_sigma'], 
      # data['features'].shape
)

[[ 0.          0.          0.        ]
 [ 0.92395162 -0.21938935  0.02704284]
 [-0.91363515 -0.20972138  0.00579915]] -75.92283626317517 0.02
